In [3]:
import os
import os.path as osp
import shutil
import numpy as np
import h5py
import torch
import pdb
from pathlib import Path
import shutil
from glob import glob

In [9]:
def xyz2rtp(xyz):
    #points [x,y,z,laserID]
    #NxMx4
    #convert points(xyz arrary) to polar coordinates
    eps = 1e-10
    rtp = np.zeros((xyz.shape[0],xyz.shape[1],3))
    #r
    rtp[:,:,0] = np.linalg.norm(xyz[:,:,:3],axis=2)
    #theta
    rtp[:,:,1] = np.arctan(xyz[:,:,1]/(xyz[:,:,0]+eps))
    #phi is ralated to laserID
    #phi
    rtp[:,:,2] = np.round(np.arccos(xyz[:,:,2]/(rtp[:,:,0]+eps)),6)
    return rtp





In [10]:
root = '../bigRed_h5_pointnet'
experiment_dir = Path('../bigRed_h5_pointnet_sorted')
experiment_dir.mkdir(exist_ok=True)


In [17]:
experiment_dir

PosixPath('../bigRed_h5_pointnet_sorted')

In [11]:
txt_list = glob('../bigRed_h5_pointnet/*.txt')

In [12]:
for item in txt_list:
    shutil.copy(item, str(experiment_dir))


In [13]:
with open(os.path.join(root, "all_files.txt"), 'r') as f:
    data_list = [x.split('/')[-1] for x in f.read().split('\n')[:-1]]

In [16]:
print('Add rtp...')
for file_name in data_list:
    file_root = os.path.join(root,file_name)
    print(file_root)
    f = h5py.File(file_root, 'r+') 
    intensity = np.array(f['intensity'])
    label = np.array(f['label'])
    laserID = np.array(f['laserID'])
    xyz = np.array(f['xyz'])
    rtp = xyz2rtp(xyz)
    f.create_dataset("rtp", data=rtp)
    f.close()


../bigRed_h5_pointnet/Complex_lounge2elevation_multiPeople.h5
../bigRed_h5_pointnet/Complex_lounge2elevation_singlePerson.h5
../bigRed_h5_pointnet/Complex_office_multiPeople.h5
../bigRed_h5_pointnet/Complex_office_singlePerson.h5
../bigRed_h5_pointnet/Medium_corridor2_multiPeople.h5
../bigRed_h5_pointnet/Medium_corridor2_singlePerson.h5
../bigRed_h5_pointnet/Medium_dinninghall_multiPeople.h5
../bigRed_h5_pointnet/Medium_dinninghall_singlePerson.h5
../bigRed_h5_pointnet/Medium_lounge2elevation_multiPeople.h5
../bigRed_h5_pointnet/Medium_lounge2elevation_singlePerson.h5
../bigRed_h5_pointnet/Simple_corridor1_multiPeople.h5
../bigRed_h5_pointnet/Simple_corridor1_singlePerson.h5
../bigRed_h5_pointnet/Simple_Labonx0y0_singlePerson.h5
../bigRed_h5_pointnet/Simple_Labonx1.5y0.5_singlePerson.h5
../bigRed_h5_pointnet/Simple_Labonx1y1_multiPeople.h5
../bigRed_h5_pointnet/Simple_Labonx1y1_singlePerson.h5


In [123]:
max_group = 0
for file_name in data_list:
    file_root = os.path.join(root,file_name)
    print(file_root)
    f = h5py.File(file_root, 'r') 
    laserID = np.array(f['laserID'])
    for i in range(len(laserID)):
        number_sheet,_,bin_sheet = torch.unique(torch.tensor(laserID[i]),sorted=True, return_inverse=True, return_counts=True, dim=None)
        if(max_group)<bin_sheet.max():
            max_group = bin_sheet.max()

../bigRed_h5_pointnet/Complex_lounge2elevation_multiPeople.h5
../bigRed_h5_pointnet/Complex_lounge2elevation_singlePerson.h5
../bigRed_h5_pointnet/Complex_office_multiPeople.h5
../bigRed_h5_pointnet/Complex_office_singlePerson.h5
../bigRed_h5_pointnet/Medium_corridor2_multiPeople.h5
../bigRed_h5_pointnet/Medium_corridor2_singlePerson.h5
../bigRed_h5_pointnet/Medium_dinninghall_multiPeople.h5
../bigRed_h5_pointnet/Medium_dinninghall_singlePerson.h5
../bigRed_h5_pointnet/Medium_lounge2elevation_multiPeople.h5
../bigRed_h5_pointnet/Medium_lounge2elevation_singlePerson.h5
../bigRed_h5_pointnet/Simple_corridor1_multiPeople.h5
../bigRed_h5_pointnet/Simple_corridor1_singlePerson.h5
../bigRed_h5_pointnet/Simple_Labonx0y0_singlePerson.h5
../bigRed_h5_pointnet/Simple_Labonx1.5y0.5_singlePerson.h5
../bigRed_h5_pointnet/Simple_Labonx1y1_multiPeople.h5
../bigRed_h5_pointnet/Simple_Labonx1y1_singlePerson.h5


In [124]:
max_group

tensor(1519)

In [170]:
tensor = torch.arange(3)

In [186]:
k = torch.randperm(3)
indices = k[:2]
indices

tensor([2, 0])

In [187]:
k

tensor([2, 0, 1])

In [180]:
indices

tensor([13,  0, 22,  6, 17,  5, 16, 28, 11, 12])

In [173]:
torch.randperm(len(tensor))

tensor([3, 5, 2, 6, 8, 9, 0, 1, 7, 4])

In [161]:
tensor = torch.zeros((1,5))


In [163]:
tensor.permute(1)

TypeError: permute() missing 1 required positional arguments: "dims"

In [53]:
file_name ="Complex_lounge2elevation_multiPeople.h5"
file_root = os.path.join(root,file_name)
print(file_root)
f = h5py.File(file_root, 'r') 
intensity = np.array(f['intensity'])
label = np.array(f['label'])
laserID = np.array(f['laserID'])
xyz = np.array(f['xyz'])
rtp = np.array(f['rtp'])
f.close()

../bigRed_h5_pointnet/Complex_lounge2elevation_multiPeople.h5


In [54]:
laserID.shape

(1992, 20000)

In [89]:
def return_sort_dict(laserID):
    new_dict = []
    laserID = torch.tensor(laserID)
    for i in laserID:
        this_ring = i
        convert_idx= this_ring.sort()[1]
        new_dict.append((convert_idx).numpy())
    return np.array(new_dict)
def sort_data(data,my_dict):
    new_data = []
    for i in range(len(data)):
        new_data.append(data[i][my_dict[i]])
    return np.array(new_data)
    

In [116]:
laserID.shape

(2001, 20000)

In [121]:
max_group = 0
for i in range(len(laserID)):
    number_sheet,_,bin_sheet = torch.unique(torch.tensor(laserID[i]),sorted=True, return_inverse=True, return_counts=True, dim=None)
    if(max_group)<bin_sheet.max():
        max_group = bin_sheet.max()

In [122]:
max_group

tensor(1342)

In [113]:
number_sheet

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15.], dtype=torch.float64)

In [115]:
bin_sheet.max()

tensor(1297)

In [91]:
new_dict = return_sort_dict(laserID)

In [95]:
new_xyz = sort_data(xyz,new_dict)

In [104]:
new_laserID = sort_data(laserID,new_dict)

In [97]:
import pptk

In [105]:
v = pptk.viewer(new_xyz[100])
v.attributes(new_laserID[100])


In [106]:
v = pptk.viewer(xyz[100])
v.attributes(laserID[100])


In [94]:
new_i = []
for i in range(len(intensity)):
    print(intensity[i])
    print(intensity[i][new_dict[i]].shape)


[25. 58.  3. ... 61.  2.  1.]
(20000,)
[ 3.  1. 60. ...  8.  1.  1.]
(20000,)
[ 1.  2. 27. ... 17.  6. 13.]
(20000,)
[41.  3. 61. ...  2. 10. 34.]
(20000,)
[13.  1. 67. ... 66. 79. 15.]
(20000,)
[ 4. 72. 22. ...  2.  1. 62.]
(20000,)
[26. 25.  6. ...  2.  0. 31.]
(20000,)
[ 7.  9.  2. ... 67. 89.  2.]
(20000,)
[ 8. 48.  1. ... 52. 52.  6.]
(20000,)
[58.  2.  0. ... 81. 26.  5.]
(20000,)
[ 2. 30. 50. ... 68. 36.  5.]
(20000,)
[ 3. 18. 58. ...  5. 67. 25.]
(20000,)
[ 56.   6.  12. ...   1.   9. 133.]
(20000,)
[ 63. 100.  10. ...  80.   4.   3.]
(20000,)
[  2.  70. 100. ...   1.   1.   3.]
(20000,)
[ 2.  3.  1. ...  5. 59. 22.]
(20000,)
[ 7. 61.  1. ... 52.  7. 10.]
(20000,)
[ 9. 10.  5. ...  1.  3.  2.]
(20000,)
[31.  2.  1. ...  3. 15.  1.]
(20000,)
[86. 23. 34. ... 57. 51. 30.]
(20000,)
[28.  1.  4. ... 11.  2. 38.]
(20000,)
[ 9. 66.  3. ... 27.  3. 57.]
(20000,)
[ 1. 52. 24. ... 61.  6.  3.]
(20000,)
[54. 13. 18. ... 27.  5. 35.]
(20000,)
[ 2. 24. 24. ... 13. 65.  5.]
(20000,)
[47. 57

(20000,)
[ 7.  8.  5. ...  1. 78. 56.]
(20000,)
[ 60.   3.  58. ...   1.   3. 100.]
(20000,)
[ 71.   9.   6. ...   1.  33. 100.]
(20000,)
[54. 68.  8. ...  1.  2.  1.]
(20000,)
[  4. 100.  69. ...   1.   9.   2.]
(20000,)
[ 9. 67.  5. ...  1. 10. 10.]
(20000,)
[11. 29.  1. ... 50.  0.  3.]
(20000,)
[23. 58. 80. ...  9. 21.  2.]
(20000,)
[74. 64. 25. ...  7.  8.  4.]
(20000,)
[38.  2. 81. ...  0.  6.  7.]
(20000,)
[52. 78.  1. ...  2.  4.  6.]
(20000,)
[ 1.  5. 35. ... 75. 55.  2.]
(20000,)
[  4.  47.   2. ...   5.  36. 100.]
(20000,)
[25.  7.  5. ... 65.  1. 53.]
(20000,)
[ 2.  1.  3. ...  1.  3. 10.]
(20000,)
[ 3.  1. 15. ...  4. 30. 46.]
(20000,)
[ 9.  2.  3. ...  5. 22.  1.]
(20000,)
[95.  1. 80. ... 60.  1.  8.]
(20000,)
[ 0.  3. 88. ... 55.  7. 18.]
(20000,)
[21.  6.  3. ...  2.  9.  2.]
(20000,)
[61.  2. 11. ... 28.  1. 15.]
(20000,)
[26. 25.  1. ...  1. 57.  3.]
(20000,)
[ 4. 65. 60. ... 10.  2.  7.]
(20000,)
[ 8.  8. 78. ...  0.  6.  5.]
(20000,)
[37. 13.  1. ...  4. 34. 43.]
(

(20000,)
[60. 75.  2. ... 25. 32.  7.]
(20000,)
[54.  1. 23. ... 40.  7.  8.]
(20000,)
[ 2. 80. 60. ... 61.  1.  8.]
(20000,)
[43.  4.  8. ... 69.  4.  2.]
(20000,)
[48. 35. 43. ...  9. 47.  1.]
(20000,)
[ 6. 39. 31. ... 64. 51.  1.]
(20000,)
[70.  1. 72. ... 12.  1.  3.]
(20000,)
[ 3. 54. 56. ...  2. 36. 10.]
(20000,)
[59.  1. 29. ...  9. 16.  1.]
(20000,)
[16. 28. 45. ...  2.  1.  2.]
(20000,)
[55.  5. 38. ... 79.  1.  5.]
(20000,)
[66.  9.  7. ...  2.  6. 12.]
(20000,)
[ 3.  2.  1. ...  7.  2. 84.]
(20000,)
[ 3.  2.  3. ...  3. 16.  3.]
(20000,)
[ 2. 16.  7. ... 41.  6.  3.]
(20000,)
[25. 54.  1. ... 17. 63. 63.]
(20000,)
[50. 63.  8. ...  2.  4.  1.]
(20000,)
[11.  6.  2. ...  2.  2.  1.]
(20000,)
[61.  4.  2. ... 68. 15.  7.]
(20000,)
[60. 19. 57. ... 32.  2. 50.]
(20000,)
[  0.  50.   1. ...   3. 100.  10.]
(20000,)
[ 1. 78. 30. ... 62.  1.  2.]
(20000,)
[ 2. 10. 36. ... 37.  2.  7.]
(20000,)
[ 75. 100.  63. ...   5.  21.   2.]
(20000,)
[ 1.  7.  8. ... 24.  0.  1.]
(20000,)
[ 3.

In [34]:
intensity = np.array(f['intensity'])
label = np.array(f['label'])
laserID = np.array(f['laserID'])
xyz = np.array(f['xyz'])

In [36]:
xyz.shape

(1992, 20000, 3)

In [45]:
a = np.array(xyz[0])

In [87]:
a.shape

(20000, 3)

In [92]:
(xyz2rtp(xyz)).shape

(1992, 20000, 3)

In [89]:
(np.pi/180)*75

1.3089969389957472